# E2B Code Sandbox with Google Gemini
Execute AI-generated Python code in a secure sandbox environment.

## Setup and Imports

In [21]:
"""E2B Code Sandbox with Google Gemini."""
import warnings
warnings.filterwarnings('ignore')

import sys
import os
import base64
import pandas as pd
import time

root_dir = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
sys.path.append(root_dir)

from dotenv import load_dotenv
load_dotenv()

from e2b_code_interpreter import Sandbox
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.tools import tool
from langchain.agents import create_agent
from langchain.messages import HumanMessage
from langgraph.checkpoint.memory import InMemorySaver

## Initialize Model and Sandbox

In [22]:
model = ChatGoogleGenerativeAI(model="gemini-3-flash-preview")

checkpointer = InMemorySaver()

sbx = Sandbox.create(timeout=1200)
print("Sandbox created")

Sandbox created


In [23]:
execution = sbx.run_code('2+2')
execution

Execution(Results: [Result(4)], Logs: Logs(stdout: [], stderr: []), Error: None)

## Helper Functions

In [24]:
def get_dataset_info(file_path):
    """Get basic dataset info."""
    if file_path.endswith('.csv'):
        df = pd.read_csv(file_path, nrows=3)
    else:
        df = pd.read_excel(file_path, nrows=3)

    return f"Columns: {list(df.columns)}\nSample data:\n{df.to_string()}"

## Define Tools

In [25]:
@tool
def upload_file(local_file_path: str):
    """Upload a data file to the E2B sandbox for analysis.
    
    Args:
        local_file_path: Local path to the file (e.g., "./data/IMDB-Movie-Data.csv")
        
    Returns:
        Success message with sandbox_path and dataset_info
    """
    if local_file_path.startswith('/'):
        local_file_path = local_file_path.lstrip('/')
    
    if not local_file_path.startswith('data/') and not local_file_path.startswith('./data/'):
        local_file_path = f"./data/{local_file_path}"
    
    if not os.path.exists(local_file_path):
        return f"Error: File not found at {local_file_path}"
    
    filename = os.path.basename(local_file_path)
    
    with open(local_file_path, "rb") as f:
        sandbox_file = sbx.files.write(f"data/{filename}", f)
    
    dataset_info = get_dataset_info(local_file_path)
    
    return f"File uploaded successfully!\nSandbox path: {sandbox_file.path}\n{dataset_info}"

@tool
def run_python_code(code: str):
    """Execute Python code in E2B sandbox.
    
    Args:
        code: Valid executable Python code. Use print() for output, display(plt.gcf()) for plots.
        
    Returns:
        Execution result
    """
    print('Running code in sandbox....')
    execution = sbx.run_code(code)
    print('Code execution finished!')

    if execution.error:
        return f"Error: {execution.error.name}\nValue: {execution.error.value}"

    os.makedirs('images', exist_ok=True)
    
    output = []
    timestamp = int(time.time())

    output.append(str(execution))
    
    # Save images
    for idx, result in enumerate(execution.results):
        if result.png:
            filename = f'images/{timestamp}_chart-{idx}.png'
            with open(filename, 'wb') as f:
                f.write(base64.b64decode(result.png))
            output.append(f'Chart saved to {filename}')


    # print('Output:', output)
    
    return "\n".join(output) if output else "Code executed successfully"

## Create Agent and Execute Query

In [26]:
system_prompt = """You are a data analysis assistant. You MUST use the available tools to complete tasks.

AVAILABLE TOOLS:
1. glob_search - Search for files in LOCAL filesystem only (searches ./data directory on your machine)
2. upload_file - Upload files from local to sandbox
3. run_python_code - Execute Python code in sandbox environment

FILE LOCATIONS:
- Local files: Use glob_search to find files in ./data directory
- Sandbox files: After upload, files are stored in '/home/user/data/' directory in code environment
- To check sandbox files: Use run_python_code with 'import os; print(os.listdir("/home/user/data/"))'

WORKFLOW - Follow these steps:
1. Search for data files using glob_search (for LOCAL file discovery only)
2. Upload file using upload_file (transfers from local to sandbox)
3. Generate and execute Python code using run_python_code (all code runs in sandbox where files are in '/home/user/data/')

VISUALIZATION RULES:
- Only create plots if user explicitly asks for: "plot", "chart", "graph", "visualize", "show"
- If plots requested: use matplotlib, add title, axis labels, display(plt.gcf())
- Otherwise: use print() for results

CRITICAL: You MUST call the appropriate tool for each step. Do not just think - ACT by calling tools."""


In [27]:
from langchain.agents.middleware import FilesystemFileSearchMiddleware, TodoListMiddleware

agent = create_agent(
    model=model,
    tools=[upload_file, run_python_code],
    system_prompt=system_prompt,
    checkpointer=checkpointer,
    middleware=[FilesystemFileSearchMiddleware(
        root_path="./data",
        use_ripgrep=True,
        max_file_size_mb=100
    )]
)



In [28]:
config = {"configurable": {"thread_id": "movie-1"}}

query = "Upload IMDB-Movie-Data.csv and create a line chart showing average ratings over years"

result = agent.invoke({"messages": [HumanMessage(content=query)]}, config=config)

response = result['messages'][-1].text
print(f"\nResponse:\n{response}")

Running code in sandbox....
Code execution finished!

Response:
The file `IMDB-Movie-Data.csv` has been uploaded, and the line chart showing the average movie ratings from 2006 to 2016 is displayed below.

The chart illustrates how the average IMDb rating has fluctuated over the decade, with a notable peak around 2007 and a general downward trend towards 2016.


In [29]:
query = "show me how many files are there? print all files."

result = agent.invoke({"messages": [HumanMessage(content=query)]}, config=config)
print(result['messages'][-1].text)

Running code in sandbox....
Code execution finished!
There is **1** file in the sandbox environment:

- `IMDB-Movie-Data.csv`


In [30]:
config = {"configurable": {"thread_id": "finance-1"}}

query = "Analyze apple 2024 and calculate financial ratios"

result = agent.invoke({"messages": [HumanMessage(content=query)]}, config=config)
print(result['messages'][-1].text)

Running code in sandbox....
Code execution finished!
Running code in sandbox....
Code execution finished!
Based on the financial statements for **Apple Inc.** for the fiscal year ended September 30, 2024, here is the analysis of its financial performance and key ratios:

### **1. Liquidity Ratios**
These ratios measure Apple's ability to cover its short-term obligations.
*   **Current Ratio: 0.87** – Apple's current assets are slightly less than its current liabilities. While a ratio below 1.0 might suggest liquidity pressure, Apple's high cash generation often allows it to operate with a lower current ratio.
*   **Quick Ratio: 0.83** – This excludes inventory, showing that Apple relies very little on selling stock to meet short-term needs.
*   **Cash Ratio: 0.17** – Approximately 17% of current liabilities can be covered immediately by cash and equivalents.

### **2. Profitability Ratios**
Apple remains highly profitable with strong margins.
*   **Gross Margin: 46.21%** – Indicates a 

In [31]:
query = "analyze google cashflow for 2024 and apple cashflow 2024. compare financial ratios. also show comparison table"

result = agent.invoke({"messages": [HumanMessage(content=query)]}, config=config)
print(result['messages'][-1].text)

Running code in sandbox....
Code execution finished!
The following analysis compares the financial performance and cash flow of **Apple Inc.** (FY 2024, ended Sept) and **Alphabet Inc. (Google)** (FY 2024, ended Dec).

### **1. Cash Flow Analysis (2024)**

*   **Apple (AAPL):**
    *   **Operating Cash Flow:** $118.25 Billion
    *   **Capital Expenditures:** ~$9.45 Billion
    *   **Free Cash Flow (FCF):** $108.81 Billion
    *   **FCF Margin:** 27.83%
    *   *Analysis:* Apple is an extremely efficient cash generator. It converts a high percentage of its revenue into free cash flow due to its relatively low capital intensity (asset-light manufacturing model).

*   **Google (GOOGL):**
    *   **Operating Cash Flow:** $125.30 Billion
    *   **Capital Expenditures:** ~$52.54 Billion
    *   **Free Cash Flow (FCF):** $72.76 Billion
    *   **FCF Margin:** 20.79%
    *   *Analysis:* While Google generates more cash from its operations than Apple, it invests significantly more in infrastr

In [32]:
query = """Analyze the 2024 cash flow and financial performance of Google (Alphabet) and Apple.
Compute key financial ratios for both companies, including Gross Profit Margin,
Net Profit Margin, ROA, ROE, Current Ratio, and Debt-to-Equity Ratio.

Create the following visualizations:
1. A grouped bar chart comparing Apple vs Google for each financial ratio
   (x-axis: ratio names, y-axis: ratio values, separate bars for Apple and Google).
2. A comparison table summarizing all calculated ratios side by side.

Add clear titles, axis labels, legends, and use distinct colors for each company.
After generating the plots and table, interpret what the visual comparison reveals
about profitability, efficiency, liquidity, and leverage differences between Apple
and Google in 2024.
"""

config = {"configurable": {"thread_id": "finance-plots-1"}}

result = agent.invoke({"messages": [HumanMessage(content=query)]}, config=config)
print(result['messages'][-1].text)

Running code in sandbox....
Code execution finished!
Running code in sandbox....
Code execution finished!
Running code in sandbox....
Code execution finished!
Running code in sandbox....
Code execution finished!
Running code in sandbox....
Code execution finished!
Based on the 2024 financial statements for **Google (Alphabet)** and **Apple**, here is a detailed analysis of their performance and financial ratios.

### **1. Key Financial Ratios Comparison**

| Financial Ratio | Google (Alphabet) | Apple |
| :--- | :--- | :--- |
| **Gross Profit Margin** | 58.20% | 46.21% |
| **Net Profit Margin** | 28.60% | 23.97% |
| **ROA (Return on Assets)** | 22.24% | 25.68% |
| **ROE (Return on Equity)** | 30.80% | 164.59% |
| **Current Ratio** | 1.84 | 0.87 |
| **Debt-to-Equity Ratio** | 0.03 | 1.87 |

---

### **2. Visual Comparison**

The following grouped bar chart compares the performance of both companies across these six key metrics.

![Financial Ratio Comparison](https://e2b-v1-production-us

In [33]:
query = """How is Alphabet funding its investing cash outflows: operating cash, debt, or cash reserves? Make sure to keep the currency denomination in your final answer
"""

result = agent.invoke({"messages": [HumanMessage(content=query)]}, config=config)
print(result['messages'][-1].text)

Running code in sandbox....
Code execution finished!
In 2024, Alphabet (Google) funded its investing cash outflows entirely through its **Operating Cash Flow**. Based on the financial statements, here is the breakdown of the cash movement (all values in **USD**):

### **1. Investing Cash Outflow**
Alphabet's total cash used for investing activities was **$45,536,000,000 USD**. 
*   This was primarily driven by **$52,535,000,000 USD** in Capital Expenditures (Purchase of PPE) and **$2,931,000,000 USD** for Acquisitions (Purchase of Business).
*   This was partially offset by a net inflow of **$12,597,000,000 USD** from the sale/maturity of investment securities.

### **2. Sources of Funding**
*   **Operating Cash Flow (Primary Source):** Alphabet generated **$125,299,000,000 USD** in cash from its core operations. This massive surplus was more than **2.7 times** the amount needed to cover all investing activities ($45.5B).
*   **Debt (Minimal Impact):** While Alphabet issued **$13,589,0

In [34]:
query = """
Using titanic.csv, calculate the survival rate for each passenger class (Pclass) and create a bar chart where 
the x-axis is Pclass (1, 2, 3) and the y-axis is survival rate (percentage of passengers who survived). 
Label the axes clearly and add a title explaining the insight.
"""

config = {"configurable": {"thread_id": "titanic-1"}}

result = agent.invoke({"messages": [HumanMessage(content=query)]}, config=config)
print(result['messages'][-1].text)

Running code in sandbox....
Code execution finished!
The survival rates for each passenger class on the Titanic are as follows:

*   **1st Class (Pclass 1):** 62.96%
*   **2nd Class (Pclass 2):** 47.28%
*   **3rd Class (Pclass 3):** 24.24%

The bar chart below visualizes these rates, showing a clear trend where passengers in higher classes had significantly higher survival rates.

![Survival Rate by Passenger Class](images/1769243039_chart-0.png)


In [35]:
query = """
From titanic.csv, group passengers by Sex and Survived, 
then create a grouped bar chart where the x-axis is Sex (male, female), 
the y-axis is passenger count, and bars are split by survival status (0 = died, 1 = survived). 
Include a legend and interpret the result.
"""

result = agent.invoke({"messages": [HumanMessage(content=query)]}, config=config)
print(result['messages'][-1].text)

Running code in sandbox....
Code execution finished!
The analysis of survival counts by sex reveals a stark difference in outcomes:

*   **Females:** 233 survived while 81 died.
*   **Males:** 109 survived while 468 died.

**Interpretation:**
The visualization clearly shows that women had a much higher probability of survival than men. This aligns with the historical "women and children first" policy during the Titanic disaster. While the total number of male passengers was significantly higher, the vast majority of them perished, whereas the majority of female passengers survived.

![Passenger Survival Count by Sex](images/1769243053_chart-0.png)


In [36]:
query = """
Using the Fare column from titanic.csv, create a box plot comparing 
ticket fares of survivors and non-survivors, with Survived (0 = did not survive, 1 = survived)
 on the x-axis and Fare on the y-axis. Apply a logarithmic scale to the Fare axis 
 if the values are highly skewed. Use distinct colors for each survival group, 
 add clear axis labels, and set the plot title to “Ticket Fare Distribution by Survival 
 Status on the Titanic”. After generating the plot, briefly explain what it reveals about 
 the relationship between socioeconomic status and survival probability.
"""

result = agent.invoke({"messages": [HumanMessage(content=query)]}, config=config)
print(result['messages'][-1].text)